Colab 기준으로 작성된 코드이며,
bert_naver_movie_colab_ipynb(https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x) 코드를 바탕으로 작성되었습니다(데이터 전처리, 테스트 부분을 제외하고는 동일).

## 1.Colab 환경 설정

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentncepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.6 MB/s eta 0:00:00


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ky_yd8si
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ky_yd8si
     |████████████████████████████████| 132 kB 24.2 MB/s 
     |████████████████████████████████| 4.5 MB 45.4 MB/s 
     |████████████████████████████████| 4.7 MB 59.6 MB/s 
     |████████████████████████████████| 6.6 MB 59.2 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 9.0 MB 71.9 MB/s 
     |████████████████████████████████| 127 kB 65.7 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=db086adf6a04eef25873dd3f6613196eedd51ea5343465a5ff46eaa7f30281f6
  Stored in director

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

1
Tesla T4


In [ ]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


## 2.데이터셋 불러오기

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
sentences = pd.read_excel('한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
chatbot_data.shape

(38594, 7)

In [ ]:
chatbot_data.sample(n=5)
from sklearn.model_selection import train_test_split

In [ ]:
sentences.loc[(sentences['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
sentences.loc[(sentences['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
sentences.loc[(sentences['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
sentences.loc[(sentences['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
sentences.loc[(sentences['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
sentences.loc[(sentences['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
sentences.loc[(sentences['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(sentences['Sentence'], sentences['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


## 3.Train data & Test data

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

28945
9649


## 4.KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 5.모델 생성

In [ ]:
class FeelingClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(FeelingClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = FeelingClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## 6.모델 학습

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/453 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.030384063720703 train acc 0.125
epoch 1 batch id 201 loss 0.6561287641525269 train acc 0.5910292288557214
epoch 1 batch id 401 loss 0.684671938419342 train acc 0.6723815461346634
epoch 1 train acc 0.679798078171666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.5330217765914314


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1878150999546051 train acc 0.9375
epoch 5 batch id 201 loss 0.21970713138580322 train acc 0.9312810945273632
epoch 5 batch id 401 loss 0.2157701700925827 train acc 0.9398768703241895
epoch 5 train acc 0.9408112582781457


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 5 test acc 0.5302511319097175


## 7.테스트셋 평가

In [ ]:
state = {
    0: '진정',
    1: '진정',
    2: '스트레스',
    3: '우울',
    6: '스트레스'
}

df = pd.read_csv('/content/drive/MyDrive/food_feeling.csv')
chatbot_data = pd.read_csv('/content/drive/MyDrive/TB_RECIPE_SEARCH-210630.csv', encoding='CP949')
food_data = chatbot_data.loc[:,['CKG_NM','CKG_MTRL_CN']]
food_data = food_data.fillna(0)

In [ ]:
def find_ingredient(df,Feel):
  sence=df[df['감정'].isin([Feel])]
  sampling=sence.sample(n =1)
  ingredient = sampling.iloc[0,1]
  element = sampling.iloc[0,2]
  return ingredient, element

In [ ]:
def get_feeling(label):
  #공포 놀람
  if label == 0 or label == 1 :
    a,b=find_ingredient(df,state[label])

  #분노 혐오
  elif label == 2 or label == 6:
    a,b=find_ingredient(df,state[label])

  #슬픔
  elif label == 3:
    a,b=find_ingredient(df,state[label])

  #행복
  elif label == 5:
    print("오늘은 가장 좋아하는 음식을 드시는건 어떤가요?")
    a = None
    b = None

  #중립
  else:
    print("추천할 음식이 없습니다.")
    a = None
    b = None

  return a, b

In [ ]:
def select_food(feel, ingredient, component):
  result = food_data.loc[food_data['CKG_MTRL_CN'].str.contains(ingredient, na=False)]
  sample_number = 100 if len(result) > 100 else len(result)
  selected_samples = result.sample(n=sample_number)

  row_index = 0
  ingr_index = 100

  for idx in selected_samples.index:
      ingredients = selected_samples.at[idx, 'CKG_MTRL_CN']
      ingredients = ingredients.lstrip("[재료] ")
      split_ingr = ingredients.split("| ")
      for i in range(len(split_ingr)):
        if ingredient in split_ingr[i]:
          if i < ingr_index:
            ingr_index = i
            row_index = idx
          break
  print(f"{ingredient}는 {component}이/가 있어 {feel}에 도움이 됩니다.")
  print(f"{selected_samples.at[row_index, 'CKG_NM']}을/를 요리하거나 사 먹어보는 것은 어떠신가요?")


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def recommend(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        number = np.argmax(logits)
        ingredient, component = get_feeling(number)
        if ingredient is not None:
          select_food(state[number], ingredient, component)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 8.새로운 문장 테스트

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("떠오르는 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

떠오르는 말을 입력해주세요 : 아무도 날 안 거드렸으면 좋겠다


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 분노가 느껴집니다.
우유는 비타민D, 트립토판이/가 있어 스트레스에 도움이 됩니다.
우유튀김을/를 요리하거나 사 먹어보는 것은 어떠신가요?


떠오르는 말을 입력해주세요 : 아무도 날 안 거드렸으면 좋겠다
>> 입력하신 내용에서 분노가 느껴집니다.
브로콜리는 칼륨, 엽산이/가 있어 스트레스에 도움이 됩니다.
투움바파스타을/를 요리하거나 사 먹어보는 것은 어떠신가요?


떠오르는 말을 입력해주세요 : 아무도 날 안 거드렸으면 좋겠다
>> 입력하신 내용에서 분노가 느껴집니다.
참외는 칼륨이/가 있어 스트레스에 도움이 됩니다.
살사을/를 요리하거나 사 먹어보는 것은 어떠신가요?


떠오르는 말을 입력해주세요 : 아무도 날 안 거드렸으면 좋겠다
>> 입력하신 내용에서 분노가 느껴집니다.
참외는 칼륨이/가 있어 스트레스에 도움이 됩니다.
참외무침을/를 요리하거나 사 먹어보는 것은 어떠신가요?


떠오르는 말을 입력해주세요 : 아싸 방학이다
>> 입력하신 내용에서 행복이 느껴집니다.
오늘은 가장 좋아하는 음식을 드시는건 어떤가요?


떠오르는 말을 입력해주세요 : 어우 짜증나
>> 입력하신 내용에서 혐오가 느껴집니다.
캐슈너트는 아연이/가 있어 스트레스에 도움이 됩니다.
멸치볶음을/를 요리하거나 사 먹어보는 것은 어떠신가요?




KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), "/content/model")

In [ ]:
model = BERTClassifier(bertmodel)
model.load_state_dict(torch.load("/content/drive/MyDrive/챗봇/model"))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(num_epochs,loss_list)
plt.subplot(1,2,2)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(num_epochs, accuracy_list)
plt.show()